### **Imports**

In [1]:
from file_io import load
from joblib import load as load_prob
from scipy.stats import wilcoxon
import numpy as np
import pylab as pl
pl.gray()

### **Auxiliary Function**

_ Inputs_

* __pred -__ the  binary segmentation;
* __gt -__ the ground truth; 
* __mask -__ the binary mask; 
* __ prob -__ the probability map (if available).

_ Outputs _

* __sen -__ Sensibility;
* __spec -__ Specificity; 
* __acc -__ Accuracy; 
* __ auc -__ Area Under the ROC curve.

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score

def perform_metrics (pred, gt, mask, prob = []):
    
    # Suppressing background regions.
    pred = pred[mask > 0]
    gt = gt[mask > 0]

    # Building confusion matrix.
    # # Cij is the number of observations known to be in group i but predicted to be in group j.
    c_matrix = confusion_matrix(gt, pred)
    
    # Calculating ratios.
    tn = c_matrix[0,0]
    tp = c_matrix[1,1]
    fn = c_matrix[1,0]
    fp = c_matrix[0,1]
    
    # Finding the metrics.
    sen  = tp/(tp + fn)
    spec = tn/(tn + fp)
    acc  = (tp + tn)/(tp + tn + fp + fn)
    
    if len (prob) != 0:
        
        prob = prob[mask > 0]
        auc  = roc_auc_score (gt, prob)
        return sen, spec, acc, auc
    
    return sen, spec, acc

### **Loading**

In [3]:
test_cases = range(1,21)
r_width, r_height = 565, 584

In [4]:
gts = np.zeros((len(test_cases), r_height, r_width))
masks = np.zeros ((len(test_cases), r_height, r_width))

In [5]:
# Nguyen (2013)
segs_nguyen = np.zeros((len(test_cases), r_height, r_width))
probs_nguyen = np.zeros((len(test_cases), r_height, r_width))

In [6]:
# Azzopardi (2015)
segs_azzopardi = np.zeros((len(test_cases), r_height, r_width))

In [7]:
# Roychowdhury (2015b)
segs_roychowdhury_b = np.zeros((len(test_cases), r_height, r_width))

In [8]:
# Niemeijer (2004) 
segs_niemeijer = np.zeros((len(test_cases), r_height, r_width))
probs_niemeijer = np.zeros((len(test_cases), r_height, r_width))

In [9]:
# Staal (2004) 
segs_staal = np.zeros((len(test_cases), r_height, r_width))
probs_staal = np.zeros((len(test_cases), r_height, r_width))

In [10]:
# Marin (2011)
segs_marin = np.zeros((len(test_cases), r_height, r_width))

In [11]:
# Roychowdhury (2015a)
segs_roychowdhury_a = np.zeros((len(test_cases), r_height, r_width))
probs_roychowdhury_a = np.zeros((len(test_cases), r_height, r_width))

In [12]:
# Liskowsi (2016)
segs_liskowski = np.zeros((len(test_cases), r_height, r_width))
probs_liskowski = np.zeros((len(test_cases), r_height, r_width))

In [13]:
# Fu (2016)
segs_fu = np.zeros((len(test_cases), r_height, r_width))
probs_fu = np.zeros((len(test_cases), r_height, r_width))

In [14]:
# Orlando (2017)
segs_orlando = np.zeros((len(test_cases), r_height, r_width))

In [15]:
# This work (2018)
segs = np.zeros((len(test_cases), r_height, r_width))
probs = np.zeros((len(test_cases), r_height, r_width))

In order for you to run the cell below, please download the resources from the links below and change the paths accordingly.

* [Nguyen (2013)](https://people.eng.unimelb.edu.au/thivun/projects/retinal_segmentation/)
* [Azzopardi (2015)](http://www.cs.rug.nl/~nick/results/)
* [Roychowdhury (2015b)](https://sites.google.com/a/uw.edu/src/useful-links)
* [Niemeijer (2004)](http://www.isi.uu.nl/Research/Databases/DRIVE/browser.php)
* [Staal (2004)](http://www.isi.uu.nl/Research/Databases/DRIVE/browser.php)
* [Marin (2011)](http://www.uhu.es/retinopathy/eng/bd.php)
* [Roychowdhury (2015a)](https://sites.google.com/a/uw.edu/src/useful-links)
* [Liskowsi (2016)](http://www.cs.put.poznan.pl/kkrawiec/?n=Site.2015IEEETMI)
* [Fu (2016)](http://hzfu.github.io/proj_deepvessel.htmlSH)
* [Orlando (2017)](http://homes.esat.kuleuven.be/~mblaschk/projects/retina/)

In [16]:
#Change these paths
gts_path = '../resources/gts/DRIVE/'
masks_path = '../resources/masks/DRIVE/'
nguyen_path = '../resources/nguyen_2013/DRIVE/'
azzopardi_path = '../resources/azzopardi_2015/DRIVE/'
roychowdhury_b_path = '../resources/roychowdhury_2015b/DRIVE/'
niemeijer_path = '../resources/niemeijer_2004/DRIVE/'
staal_path = '../resources/staal_2004/DRIVE/'
marin_path = '../resources/marin_2011/DRIVE/'
roychowdhury_a_path = '../resources/roychowdhury_2015a/DRIVE/'
liskowski_path = '../resources/liskowski_2016/DRIVE/'
fu_path = '../resources/fu_2016/DRIVE/'
orlando_path = '../resources/orlando_2017/DRIVE/'

for i, case in enumerate(test_cases):
    
    # Ground-truth and masks
    gt = load(gts_path + '%02d_manual1.gif' %case, normalize = True)
    gts[i] = gt
    mask = load(masks_path + '%02d_test_mask.gif' %case)
    masks[i] = mask
    
    # Nguyen (2013)
    seg_nguyen = load(nguyen_path + 'drive_%02d_test_multi_scales_unsupervised.png' %case, normalize = True)
    segs_nguyen[i] = seg_nguyen
    prob_nguyen = load(nguyen_path + 'drive_%02d_test_multi_scales_unsupervised_soft.png' %case, normalize = True)
    probs_nguyen[i] = prob_nguyen
    
    # Azzopardi (2015)
    seg_azzopardi = load(azzopardi_path + '%02d_test.gif' %case, normalize = True)[:,:,0]
    segs_azzopardi[i] = seg_azzopardi
    
    # Roychowdhury (2015b)
    seg_roychowdhury_b = load(roychowdhury_b_path + '%02d_test.tif' %case, normalize = True)
    segs_roychowdhury_b[i] = seg_roychowdhury_b
    
    # Niemeijer (2004)
    seg_niemeijer = load(niemeijer_path + '%02d_PC_hard.gif' %case, normalize = True)
    segs_niemeijer[i] = seg_niemeijer
    prob_niemeijer = load(niemeijer_path + '%02d_PC_soft.gif' %case, normalize = True)
    probs_niemeijer[i] = prob_niemeijer
    
    # Staal (2004)
    seg_staal = load(staal_path + '%02d_Staal_hard.gif' %case, normalize = True)[:,:,0]
    segs_staal[i] = seg_staal
    prob_staal = load(staal_path + '%02d_Staal_soft.gif' %case, normalize = True)
    probs_staal[i] = prob_staal
    
    # Marin (2011)
    seg_marin = load(marin_path + 'Im%02d.tif' %case, normalize = True)
    segs_marin[i] = seg_marin
    
    # Roychowdhury (2015a)
    seg_roychowdhury_a = load(roychowdhury_a_path + '%02d_test.tif' %case, normalize = True)
    segs_roychowdhury_a[i] = seg_roychowdhury_a
    
    # Liskowski (2016)
    prob_liskowski = 1 - load(liskowski_path + '%02d_test.png' %case, normalize = True)[27:611, 27:592]
    probs_liskowski[i] = prob_liskowski
    seg_liskowski = np.where(prob_liskowski < 0.5, 0, 1)
    segs_liskowski[i] = seg_liskowski
    
    # Fu (2016)
    seg_fu = load(fu_path + '%02d_test_seg_result.png' %case, normalize = True)
    segs_fu[i] = seg_fu
    prob_fu = load(fu_path + '%02d_test_CRF_result.png' %case, normalize = True)
    probs_fu[i] = prob_fu
    
    # Orlando (2017)
    seg_orlando = load(orlando_path + '%02d_test_fccrf.png' %case, normalize = True)
    segs_orlando[i] = seg_orlando
    
    # This work (2018)
    seg = load('../resources/binary_segmentations/DRIVE/seg_%04d.png' %case, normalize = True)
    prob = load_prob('../resources/probability_maps/DRIVE/prob_%04d.npy' %case)[1] 
    segs[i] = seg
    probs[i] = prob

### **Calculating metrics**

In [17]:
def get_metrics (segs, gts, masks, probs = []):

    if len(probs) != 0:

        sen_list, spec_list, acc_list, auc_list = [], [], [], []

        for i in range (len(gts)):
            sen, spec, acc, auc = perform_metrics (segs[i], gts[i], masks[i], probs[i])
            sen_list.append(sen)
            spec_list.append(spec)
            acc_list.append(acc)
            auc_list.append(auc)
            
        return sen_list, spec_list, acc_list, auc_list
            
    else:
    
        sen_list, spec_list, acc_list = [], [], []

        for i in range (len(gts)):
            sen, spec, acc = perform_metrics (segs[i], gts[i], masks[i])
            sen_list.append(sen)
            spec_list.append(spec)
            acc_list.append(acc)
            
        return sen_list, spec_list, acc_list


In [18]:
sen_list_nguyen, spec_list_nguyen, acc_list_nguyen, auc_list_nguyen = get_metrics (segs_nguyen, gts, masks, probs_nguyen)

In [19]:
sen_list_azzopardi, spec_list_azzopardi, acc_list_azzopardi = get_metrics (segs_azzopardi, gts, masks)

In [20]:
segs_roychowdhury_b[2] = np.where(segs_roychowdhury_b[2] > 0.5, 1, 0)
sen_list_roychowdhury_b, spec_list_roychowdhury_b, acc_list_roychowdhury_b = get_metrics (segs_roychowdhury_b, gts, masks)

In [21]:
for i in range(len(gts)): 
    segs_niemeijer[i] = np.where(segs_niemeijer[i] >= 0.5, 1, 0)

In [22]:
sen_list_niemeijer, spec_list_niemeijer, acc_list_niemeijer, auc_list_niemeijer = get_metrics (segs_niemeijer, gts, masks, probs_niemeijer)

In [23]:
for i in range(len(gts)): 
    segs_staal[i] = np.where(segs_staal[i] >= 0.5, 1, 0)

In [24]:
sen_list_staal, spec_list_staal, acc_list_staal, auc_list_staal = get_metrics (segs_staal, gts, masks, probs_staal)

In [25]:
sen_list_marin, spec_list_marin, acc_list_marin = get_metrics (segs_marin, gts, masks)

In [26]:
for i in range(len(gts)): 
    segs_roychowdhury_a[i] = np.where(segs_roychowdhury_a[i] >= 0.5, 1, 0)

In [27]:
sen_list_roychowdhury_a, spec_list_roychowdhury_a, acc_list_roychowdhury_a = get_metrics (segs_roychowdhury_a, gts, masks)

In [28]:
sen_list_liskowski, spec_list_liskowski, acc_list_liskowski, auc_list_liskowski = get_metrics (segs_liskowski, gts, masks, probs_liskowski)

In [29]:
sen_list_fu, spec_list_fu, acc_list_fu, auc_list_fu = get_metrics (segs_fu, gts, masks, probs_fu)

In [30]:
sen_list_orlando, spec_list_orlando, acc_list_orlando = get_metrics (segs_orlando, gts, masks)

In [31]:
sen_list, spec_list, acc_list, auc_list = get_metrics (segs, gts, masks, probs)

### **Making Statistical Comparison**

In [32]:
# Nguyen (2013)
_, pvalue = wilcoxon(acc_list_nguyen, acc_list)
pvalue

8.8574576878635474e-05

In [33]:
# Azzopardi (2015)
_, pvalue = wilcoxon(sen_list_azzopardi, sen_list)
pvalue

0.0071891683913754955

In [34]:
_, pvalue = wilcoxon(spec_list_azzopardi, spec_list)
pvalue

0.0015072868413933233

In [35]:
_, pvalue = wilcoxon(acc_list_azzopardi, acc_list)
pvalue

8.8574576878635474e-05

In [36]:
auc_list_azzopardi = [0.9736, 0.9714, 0.9511, 0.9567, 0.9537, 
                      0.9520, 0.9519, 0.9466, 0.9576, 0.9601, 
                      0.9515, 0.9629, 0.9563, 0.9682, 0.9651,
                      0.9712, 0.9574, 0.9685, 0.9774, 0.9742]

In [37]:
_, pvalue = wilcoxon(auc_list_azzopardi, auc_list)
pvalue

8.8574576878635474e-05

In [38]:
# Roychowdhury (2015b)
_, pvalue = wilcoxon(sen_list_roychowdhury_b, sen_list)
pvalue

8.8574576878635474e-05

In [39]:
_, pvalue = wilcoxon(spec_list_roychowdhury_b, spec_list)
pvalue

0.00021907963048002185

In [40]:
_, pvalue = wilcoxon(acc_list_roychowdhury_b, acc_list)
pvalue

8.8574576878635474e-05

In [41]:
# Niemeijer (2004)
_, pvalue = wilcoxon(acc_list_niemeijer, acc_list)
pvalue

8.8574576878635474e-05

In [42]:
_, pvalue = wilcoxon(auc_list_niemeijer, auc_list)
pvalue

8.8574576878635474e-05

In [43]:
# Staal (2004)
_, pvalue = wilcoxon(acc_list_staal, acc_list)
pvalue

8.8574576878635474e-05

In [44]:
_, pvalue = wilcoxon(auc_list_staal, auc_list)
pvalue

8.8574576878635474e-05

In [45]:
# Marin (2011)
_, pvalue = wilcoxon(sen_list_marin, sen_list)
pvalue

8.8574576878635474e-05

In [46]:
_, pvalue = wilcoxon(spec_list_marin, spec_list)
pvalue

0.10842674449200547

In [47]:
_, pvalue = wilcoxon(acc_list_marin, acc_list)
pvalue

8.8574576878635474e-05

In [48]:
# Roychowdhury (2015a)
_, pvalue = wilcoxon(sen_list_roychowdhury_a, sen_list)
pvalue

8.8574576878635474e-05

In [49]:
_, pvalue = wilcoxon(spec_list_roychowdhury_a, spec_list)
pvalue

8.8574576878635474e-05

In [50]:
_, pvalue = wilcoxon(acc_list_roychowdhury_a, acc_list)
pvalue

8.8574576878635474e-05

In [51]:
# Liskowski (2016)
_, pvalue = wilcoxon(sen_list_liskowski, sen_list)
pvalue

8.8574576878635474e-05

In [52]:
_, pvalue = wilcoxon(spec_list_liskowski, spec_list)
pvalue

0.60121270573980956

In [53]:
_, pvalue = wilcoxon(acc_list_liskowski, acc_list)
pvalue

8.8574576878635474e-05

In [54]:
_, pvalue = wilcoxon(auc_list_liskowski, auc_list)
pvalue

8.8574576878635474e-05

In [55]:
# Fu (2016)
_, pvalue = wilcoxon(sen_list_fu, sen_list)
pvalue

0.00016285576606790404

In [56]:
_, pvalue = wilcoxon(acc_list_fu, acc_list)
pvalue

8.8574576878635474e-05

In [57]:
# Orlando (2017)
_, pvalue = wilcoxon(sen_list_orlando, sen_list)
pvalue

0.079321679696761765

In [58]:
_, pvalue = wilcoxon(spec_list_orlando, spec_list)
pvalue

0.0001401336206158955

In [59]:
_, pvalue = wilcoxon(acc_list_orlando, acc_list)
pvalue

8.8574576878635474e-05